In [ ]:
import import_utils
import pandas

In [ ]:
import_utils.download_and_unzip_file("ftp://ftp.snoco.org/Assessor/shapefiles/parcels.zip")

In [ ]:
!shp2pgsql -s 2285 -W latin1 -D data/parcels.shp import.snohomish_county_parcel_geo > data/snohomish_county_parcel_geo.sql
!psql -U postgres -d work -f data/snohomish_county_parcel_geo.sql

In [ ]:
!psql -U postgres -d work --tuples-only -c "select 'DROP TYPE IF EXISTS snohomish_county_use_type;  CREATE TYPE snohomish_county_use_type AS ENUM (' || array_to_string(array_agg(quote_literal(dd) ORDER BY dd), ', ') || ');' from (select distinct usecode dd from import.snohomish_county_parcel_geo) t" > sql/create_snohomish_county_enums.sql
!psql -U postgres -d work -f sql/create_snohomish_county_enums.sql


In [ ]:
import_utils.run_clean_script("snohomish_county_parcel_geo")
import_utils.run_sql("DROP TABLE import.snohomish_county_parcel_geo")
import_utils.run_command("rm data/parcels.*")
import_utils.run_command("rm data/disclaimer_termsofuse.txt")
import_utils.run_command("rm data/taxacct_document.dbf")
import_utils.run_command("rm data/parcels_data_dictionary.pdf")
import_utils.run_command("rm data/snohomish_county_parcel_geo.sql")

In [ ]:
!wget -O data/sno-2016.xlsx ftp://ftp.snoco.org/Assessor/Property_Sales/Snohomish%20County%20Sales/2016.07.21_Entire%20County.xlsx
!wget -O data/sno-2017.xlsx ftp://ftp.snoco.org/Assessor/Property_Sales/Snohomish%20County%20Sales/2017.02.01_Entire%20County.xlsx
!wget -O data/sno-2018.xlsx ftp://ftp.snoco.org/Assessor/Property_Sales/Snohomish%20County%20Sales/2018.03.20_Entire%20County.xlsx
!wget -O data/sno-2018_2.xlsx ftp://ftp.snoco.org/Assessor/Property_Sales/Snohomish%20County%20Sales/2018.10.24_Entire%20County.xlsx
!wget -O data/sno-2019.xlsx ftp://ftp.snoco.org/Assessor/Property_Sales/Snohomish%20County%20Sales/2019.05.21_Entire%20County.xlsx


In [ ]:
df = pandas.read_excel("data/sno-2016.xlsx")


In [ ]:
df.to_csv("data/sno-import.csv", index=False)

In [ ]:
import_utils.load_csv_to_postgres("snohomish_sale", "data/sno-import.csv")